In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, precision=2)

# Define goal

Our goal will be denoted as $g$, and in this case $g=(5,0)$

In [2]:
g = np.array([5, 0])

# Collect Initial Data

Let's pretend our data comes from pulling a 2-link object to the right for 6 time steps

In [3]:
s = np.array([[0,0,1,0,2,0],[1,0,2,0,3,0],[2,0,3,0,4,0],[3,0,4,0,5,0],[4,0,5,0,6,0],[5,0,6,0,7,0]])
a = np.array([[1,0], [1,0], [1,0], [1,0], [1,0], [1,0]])
D = [(s_t, a_t) for s_t, a_t in zip(s, a)]
print('s =\n', s)
print('a =\n', a)

s =
 [[0 0 1 0 2 0]
 [1 0 2 0 3 0]
 [2 0 3 0 4 0]
 [3 0 4 0 5 0]
 [4 0 5 0 6 0]
 [5 0 6 0 7 0]]
a =
 [[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


# Fit Model Reduction w/ PCA

In [4]:
s_bar = s.mean(axis=0)
z = (s - s_bar)
D, P = np.linalg.eig(z.T@z)
k=1
idx = D.argsort()[-k:][::-1]   
P_star = P[:,idx]
o = z @ P_star

print('z =\n', z)
print('P* =\n', P_star)
print("o =\n", o)

z =
 [[-2.5  0.  -2.5  0.  -2.5  0. ]
 [-1.5  0.  -1.5  0.  -1.5  0. ]
 [-0.5  0.  -0.5  0.  -0.5  0. ]
 [ 0.5  0.   0.5  0.   0.5  0. ]
 [ 1.5  0.   1.5  0.   1.5  0. ]
 [ 2.5  0.   2.5  0.   2.5  0. ]]
P* =
 [[0.58]
 [0.  ]
 [0.58]
 [0.  ]
 [0.58]
 [0.  ]]
o =
 [[-4.33]
 [-2.6 ]
 [-0.87]
 [ 0.87]
 [ 2.6 ]
 [ 4.33]]


We can see here that in fact the covariance matrix of our data has only one eigenvector with non-zero eigen values, which is what we expect because all of the coordinates can be represented as one number. The resulting number (each row in $o$) is not actually the x coordinate of the first link, but it's proportional to it. This still has the property that the dynamics are linear, which is what we want.

# Fit Linear Dynamics with Least-Squares

Our dynamics $T(o_t, a_t) \rightarrow \dot{o}_t$ take the form:

$$\begin{bmatrix}o_t & v_x & v_y\end{bmatrix} b = \dot{o}_t$$
$$\begin{bmatrix}o_t & v_x & v_y\end{bmatrix} \begin{bmatrix}b_1 \\ b_2 \\ b_3\end{bmatrix} = \dot{o}_t$$

We can find the least squares solution to this. Our values of $\dot{o}_t$ come from computing $o_{t+1} - o_t$.

In [18]:
o_dot = (o[1:] - o[:-1])
ov = np.hstack((o, a))[:-1,:]
b, res, _, __ = np.linalg.lstsq(ov, o_dot, rcond=None)
print('b =\n', b)


b =
 [[-0.  ]
 [ 1.73]
 [ 0.  ]]


# Fit the Cost Function with Least-Squares

In our toy problem the distance is just the error in x.

Our cost function estimator $c(o_t) \rightarrow \hat{c_t}$ takes the form

$$ O*C  = \hat{c_t} $$
$$ \begin{bmatrix}o_t & 1\end{bmatrix} \begin{bmatrix}c_1 \\ c_2\end{bmatrix}  = \hat{c_t} $$

In [6]:
c = np.expand_dims(np.sum(g - s[:,:2], axis=1), axis=0).T
print('c(s_t) =\n', c)
O = np.hstack((o, np.ones(o.shape)))
C, res, _, __ = np.linalg.lstsq(O, c, rcond=None)
print('C =\n', C)

c(s_t) =
 [[5]
 [4]
 [3]
 [2]
 [1]
 [0]]
C =
 [[-0.58]
 [ 2.5 ]]


as we would expect, the cost can be perfectly recovered from the presentation by computing $o_t * -0.58 + 2.5 = c(s_t)$

# Plan!

Now that we have learned representation, dynamics, and cost, we can plan to reach the goal. Because in this toy problem all of these functions are perfect (no error in predicting future representation or future cost), our plans should also be perfect. The perfect plan is to take the action of $[1, 0]$ for 5 time steps.

Because we assume know obstacles, and our dynamics are linear, we can directly solve for a one-step plan (single control action) to take us from start to goal). This is simply an under-determined linear system, where $o_t$, $b$, and $\dot{o}_t$ are known and we want to solve for $v_x$ and $v_y$.

In [20]:
s_0 = np.array([[0, 0, 1, 0, 2, 0]])
o_0 = (s_0 - s_bar) @ P_star
o_g = (g - s_bar) @ P_star
o_dot_0 = o_g - o_0

ov, res, _, __ = np.linalg.lstsq(b.T, o_dot_0, rcond=None)
print(o_dot_0)
plan = np.array([[ov[1], ov[2]]])
print(plan)

o_t = o_0
for t, planned_action in enumerate(plan):
    o_t_next = o_t + np.vstack((o_t, np.expand_dims(planned_action, axis=1))).T @ b
    c_hat_t = np.hstack((o_t_next, np.ones(o_t.shape))) @ C
    o_t = o_t_next
    print(o_t_next, c_hat_t)
    # We would execute the first action of this plan in our simulator,
    # then get a new state transition pair, re-train everything,
    # then replan, and repeat...

ValueError: operands could not be broadcast together with shapes (2,) (6,) 

Ok so this works perfectly. So now what happens when we change the goal? Let the goal have non-zero y value.

$$ g = (3,1) $$

Everything will be the same as above (our first action will be [1,0] since the goal still has positive x, and our dynamics know how to move us in x. But now when we get to the step of fitting our cost, there will be an issue. We can no longer fit a simply line to our cost if our model reduction ignores the y coordinate...

Note to self: if we did PCA on initial data where the pull was not just to the right, PCA would still try to explain the y motion a bit too so we won't get a pure, clean, reduction to $[x_1, y_1, 0, 0, 0, 0]$